# Container DAG


https://docker-py.readthedocs.io/en/stable/index.html

https://docs.docker.com/reference/cli/docker/

In [71]:
from abc import ABCMeta, abstractmethod
import json
import random
import subprocess
import sys
from dataclasses import dataclass

from docker import from_env

import nevergrad as ng
import scipy.optimize as sco
import numpy as np

In [2]:
sys.path.append("../../src")

from dockweed.topological_sort import kahns_algorithm
from dockweed.process import *
from dockweed.container import *
from dockweed.graph import Graph

## Open Docker client

In [3]:
client = from_env()

### Get container processes

In [4]:
containers = []
processes = {}
# Loop through images. 
# Exclude intermediate layer images (all=False) and dangling images.
for image in client.images.list(all=False, filters={"dangling":False}):
    
    # Image tag
    tags = image.tags
    if not tags:
        continue
    image_tag = tags[0]
    
    # Labels with node names and commands.
    labels = image.attrs['ContainerConfig']['Labels']
    if labels is None:
        continue
    process_cmd = {name: cmd for name, cmd in labels.items() if name.startswith("process.")}
    if not process_cmd:
        continue
        
    # Processes
    for name, cmd in process_cmd.items():
        # Container should have no entry point, or this will call the entrypoint rather than label command.
        result = client.containers.run(image=image_tag, command=cmd, remove=True, stdout=True)
        try:
            dic = json.loads(result)
            if "description" in dic and "outputs" in dic and "inputs" in dic:
                
                if not containers or containers[-1].image != image_tag:
                    container = NodeContainer(image_tag=tags[0])
                    containers.append(container) 
                
                processes[name] = ContainerProcess(
                    name = name,
                    description = dic["description"],
                    container = containers[-1].name,
                    command = cmd,
                    inputs = dic["inputs"],
                    outputs = dic["outputs"]
                )    
        except json.decoder.JSONDecodeError:
            continue
    
print([container.name for container in containers])
print([process.name for process in processes.values()])

['multiple_divide', 'add_subtract']
['process.c_divide', 'process.c_multiply', 'process.python_add', 'process.python_subtract']


### Start containers

In [5]:
for container in containers:
    container.start()

84afe6f2666f9b4d764cd71af72011673f920d5d2b372346a0a668eba800f46f
0a833644f95f5066e0a4024965dc5968c8fb8d28dd3a1e33a96f8a8ba4d30568


### Add generator processes

In [6]:
processes["random float"] = UniformFloatProcess()
processes["random choice"] = RandomChoice()

### Information about processes

In [7]:
for i, process in enumerate(processes.values()):
    print("-"*30)
    print(f"#{i} {process.name}")
    print(f"\t{process.description}")
    print(f"\tinputs = {process.inputs}")
    print(f"\toutputs = {process.outputs}")

------------------------------
#0 process.c_divide
	Division. z = a / b
	inputs = {'a': 1.0, 'b': 1.0}
	outputs = {'z': 1.0}
------------------------------
#1 process.c_multiply
	Multiplication. z = a * b
	inputs = {'a': 1.0, 'b': 1.0}
	outputs = {'z': 1.0}
------------------------------
#2 process.python_add
	Addition. z = x + y.
	inputs = {'x': 1.0, 'y': 1.0}
	outputs = {'z': 1.0}
------------------------------
#3 process.python_subtract
	Addition. z = x + y.
	inputs = {'x': 1.0, 'y': 1.0}
	outputs = {'z': 1.0}
------------------------------
#4 random float
	A uniformly distributed random float.
	inputs = {'min': 0.0, 'max': 1.0}
	outputs = {'n': 0.5}
------------------------------
#5 random choice
	A rnadom choice.
	inputs = {'choices': [1, 2, 3, 4]}
	outputs = {'choice': 1}


### Run individual processes

In [8]:
processes["process.python_add"].run({'x': 2.34, 'y': 10.78})

{'z': 13.12}

In [9]:
processes["process.c_multiply"].run({'a': 89.987, 'b': 10.0})

{'z': 899.8699951171875}

In [10]:
for _ in range(10):
    x = processes["random choice"].run({'choices': [3, 4, 8]})
    print(x)

{'choice': 4}
{'choice': 8}
{'choice': 3}
{'choice': 3}
{'choice': 4}
{'choice': 3}
{'choice': 4}
{'choice': 8}
{'choice': 8}
{'choice': 3}


### Run a graph of processes

#### Specify the graph

In [85]:
graph_specification = {
    "node alpha": {
        "process": "random choice",
        "inputs": {'choices': [1.2, 5.4, 6.7]}
    },
    "node beta": {
        "process": "random float",
        "inputs": {"min": -5.0, "max": 5.0}
    },
    "node a": {
        "process": "process.python_add",
        "inputs": {'x': ("node alpha", "choice"), 'y': 16.5}
    },
    "node b": {
        "process": "process.python_add",
        "inputs": {'x': ("node a", "z"), 'y': ("node beta", "n")}
    },
    "node c": {
        "process": "process.c_divide",
        "inputs": {'a': ("node a", "z"), 'b': ("node b", "z")}
    },
}

In [80]:
graph_specification = {
    "node a": {
        "process": "process.python_add",
        "inputs": {'x': 2.1, 'y': 16.5}
    },
    "node b": {
        "process": "process.python_add",
        "inputs": {'x': ("node a", "z"), 'y': 5.4}
    },
    "node c": {
        "process": "process.c_divide",
        "inputs": {'a': ("node a", "z"), 'b': ("node b", "z")}
    },
}

In [11]:
# Parabolics well. y = (x - x0)^2 + y0
graph_specification = {
    "xp": {
        "process": "process.python_subtract",
        "inputs": {'x': 1.0, 'y': 4.0}
    },
    "square": {
        "process": "process.c_multiply",
        "inputs": {'a': ("xp", "z"), 'b': ("xp", "z")}
    },
     "yp": {
        "process": "process.python_add",
        "inputs": {'x': ("square", "z"), 'y': 1.0}
    },
}

In [12]:
graph = Graph(graph_specification, list(processes.values()))

In [13]:
inputs, outputs = graph.run()
print(inputs)
print("="*20)
print(outputs)

{'xp': {'x': 1.0, 'y': 4.0}, 'square': {'a': -3.0, 'b': -3.0}, 'yp': {'x': 9.0, 'y': 1.0}}
{'xp': {'z': -3.0}, 'square': {'z': 9.0}, 'yp': {'z': 10.0}}


In [14]:
print(graph.input_value("xp", "x"))

1.0


In [83]:
@dataclass
class FreeInput:
    node: str
    variable: str
        
@dataclass
class ScalarInput(FreeInput):
    min_value: float
    max_value: float
    init_value: float
        
@dataclass
class ListInput(FreeInput):
    values: list
    init_index: int
    
    
class GraphOptimizer:
    
    def __init__(self, graph: Graph, free_inputs: list, optimize_on: tuple, minimize=True):
        
        self.graph = graph
        self.free_inputs = free_inputs
        self.optimize_on = optimize_on
        self.minimize = minimize
        errors = []
        
        # Check free inputs.
        for finput in free_inputs:
            value = self.graph.input_value(finput.node, finput.variable)
            if value is None:
                errors.append(f"Input {finput.node}, {finput.variable}: Does not exist.")
                continue
            if isinstance(value, tuple):
                errors.append(f"Input {finput.node}, {finput.variable}: Is an edge.")
                continue
            if isinstance(value, list) and isinstance(finput, ScalarInput):
                errors.append(f"Input {finput.node}, {finput.variable}: Is a list and must have a non-scalar parameterisation.")
                continue
                
            # ????
           
        # Check output to be optimised.
        node, variable = optimize_on
        value = self.graph.output_value(node, variable)
        if value is None:
            errors.append(f"Output {node}, {variable}: Does not exist.")
        elif not isinstance(value, (int, float)):
            errors.append(f"Output {node}, {variable}: Must a real number to optimise.")
            
        if errors:
            raise KeyError("\n".join(errors))
            
    
    def run(self, *vargs):
        # Set graph inputs.
        for i, finput in enumerate(self.free_inputs):
            self.graph.specification[finput.node]["inputs"][finput.variable] = vargs[i]  # Have set input func for graph.
        # Run graph.
        return self.graph.run()         
        
    def objective(self, *vargs):
        
        _, outputs = self.run(*vargs)
        # Return optimized output.
        node, variable = self.optimize_on
        value = outputs[node][variable]
        if not self.minimize:
            value *= -1
        return value   
    
    def objective_array(self, x):
        return self.objective(*x.tolist())
        
    def optimize_nevergrad(self, optimizer_class, **optimizer_kwargs):
        
        params = []
        for finput in self.free_inputs:
            if isinstance(finput, ScalarInput):
                params.append(ng.p.Scalar(
                    init=finput.init_value, 
                    lower=finput.min_value, 
                    upper=finput.max_value
                ))
            elif isinstance(finput, ListInput):
                params.append(ng.p.Choice(
                    choices=finput.values,
                ))
            else:
                raise ValueError("Must be a scalar or list input.")

        optimizer = optimizer_class(
            parametrization=ng.p.Instrumentation(*params), 
            **optimizer_kwargs
        )
        solution = optimizer.minimize(self.objective).value[0]
        return self.run(*solution)
    
    def optimize_scipy(self, method: str, **optimizer_kwargs):
        
        x0 = []
        bounds = []
        for finput in self.free_inputs:
            if isinstance(finput, ScalarInput):
                x0.append(finput.init_value)
                bounds.append((finput.min_value, finput.max_value))
            else:
                raise ValueError("Must be a scalar or list input.")

        solution = sco.minimize(fun=self.objective_array, x0=np.array(x0), method=method, bounds=bounds)
        return self.run(*solution.x.tolist())    

In [84]:
nop = GraphOptimizer(
    graph = graph,
    free_inputs = [
        ScalarInput(node="xp", variable="x", init_value=2.1, min_value=-5, max_value=5), 
    ],
    optimize_on = ("yp", "z"),
    minimize=True
)

In [85]:
nop.optimize_scipy(method="nelder-mead")

({'xp': {'x': 3.9999975585937486, 'y': 4.0},
  'square': {'a': -2.4414062513855583e-06, 'b': -2.4414062513855583e-06},
  'yp': {'x': 5.960464326965065e-12, 'y': 1.0}},
 {'xp': {'z': -2.4414062513855583e-06},
  'square': {'z': 5.960464326965065e-12},
  'yp': {'z': 1.0000000000059606}})

In [86]:
nop.optimize_nevergrad(ng.optimization.optimizerlib.NGO, budget=100)

({'xp': {'x': 4.000157661700929, 'y': 4.0},
  'square': {'a': 0.00015766170092934573, 'b': 0.00015766170092934573},
  'yp': {'x': 2.4857213176687765e-08, 'y': 1.0}},
 {'xp': {'z': 0.00015766170092934573},
  'square': {'z': 2.4857213176687765e-08},
  'yp': {'z': 1.0000000248572132}})

In [ ]:
graph_specification = {
    "xp": {
        "process": "process.python_subtract",
        "inputs": {'x': 1.0, 'y': 4.0}
    },
    "square": {
        "process": "process.c_multiply",
        "inputs": {'a': ("xp", "z"), 'b': ("xp", "z")}
    },
     "yp": {
        "process": "process.python_add",
        "inputs": {'x': ("square", "z"), 'y': 1.0}
    },
}

### Stop containers

In [87]:
for container in containers:
    container.stop()

multiple_divide
add_subtract


In [ ]:
# echo '{"x": 11.98769, "y": 186.78}' | docker exec -i some-node-a python run.py


## Close client

In [88]:
client.close()